In [12]:
import random, glob
import numpy as np

In [13]:
import numpy as np

str = "['A3.A5@1.25' 'E4.E6@2' 'E4.E6@1.25' 'D4.E4.D6.E6@0.3' 'D4.D6@1.25'\n 'C4.C6@1.25' 'D4.D6@1.25' 'E4.E6@1.25' 'F4.F6@1.25' 'G4.G6@1.25']"

# Remove square brackets and "\n" characters
str = str.replace("[", "").replace("]", "").replace("\n", "").replace("'","")
print(str)

# Split the string by space to get a list of strings
str_list = str.split(" ")
print(str_list)

# Convert the list of strings to a numpy array
result = np.array(str_list)
result



A3.A5@1.25 E4.E6@2 E4.E6@1.25 D4.E4.D6.E6@0.3 D4.D6@1.25 C4.C6@1.25 D4.D6@1.25 E4.E6@1.25 F4.F6@1.25 G4.G6@1.25
['A3.A5@1.25', 'E4.E6@2', 'E4.E6@1.25', 'D4.E4.D6.E6@0.3', 'D4.D6@1.25', 'C4.C6@1.25', 'D4.D6@1.25', 'E4.E6@1.25', 'F4.F6@1.25', 'G4.G6@1.25']


array(['A3.A5@1.25', 'E4.E6@2', 'E4.E6@1.25', 'D4.E4.D6.E6@0.3',
       'D4.D6@1.25', 'C4.C6@1.25', 'D4.D6@1.25', 'E4.E6@1.25',
       'F4.F6@1.25', 'G4.G6@1.25'], dtype='<U15')

In [14]:
from music21 import midi
def play_midi_file(midi_file_name):
    mf = midi.MidiFile()

    mf.open(midi_file_name) # path='abc.midi'
    mf.read()
    mf.close()
    s = midi.translate.midiFileToStream(mf)
    s.show('midi')

In [15]:
from music21 import converter,corpus, chord

In [16]:
from music21 import chord, duration

Classifying the music into different modes
- firstly, let's code for major mode

In [17]:
from music21 import *

In [18]:
# for seeing output from chords and given duration:
def get_music_midi_from_chords_and_duration(input_chords):
    midi_stream = stream.Stream()

    for note_pattern, duration_pattern in input_chords:
        notes_in_chord = note_pattern.split('.')
        
        chord_notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(current_note)
            new_note.duration = duration.Duration(duration_pattern)
            new_note.storedInstrument = instrument.Violoncello()
            chord_notes.append(new_note)
        new_chord = chord.Chord(chord_notes)
        
        midi_stream.append(new_chord)

        new_tempo = tempo.MetronomeMark(number=50)
            
        midi_stream.append(new_tempo)

    midi_stream = midi_stream.chordify()
    timestr = time.strftime("%Y%m%d-%H%M%S")
    new_file = 'output-' + timestr + '.mid'
    return midi_stream.write('midi', fp=new_file)

In [19]:
def round_chord_durations(number):
    if(number>1.25):
        return 2
    if(number>.30):
        return 1.25
    if(number>.10):
        return 0.3
    return 0.1

For chord and duration of a single song(music file)

In [20]:
def get_chord_and_duration_data(indv_midi_chordified):
    chord_duration_data = []
    for element in indv_midi_chordified.flat:
        if isinstance(element, chord.Chord):
            chord_duration = round_chord_durations(element.duration.quarterLength)
            chord_name = ".".join([n.nameWithOctave for n in element.pitches])
            chord_duration_data.append((chord_name, chord_duration))

    return chord_duration_data

from Tonic_mode_all, separating major and minor songs 

* Data preparation stage

In [21]:
import pandas as pd

In [60]:
df =pd.read_csv('../dataset_processed/initial_dataset_major_songs_295.csv')

In [61]:
df.columns

Index(['Unnamed: 0', 'music_number', '32_chords_and_duration',
       'target_chords_and_duration'],
      dtype='object')

In [62]:
type(df['32_chords_and_duration'][0])

str

In [63]:
df['target_chords_and_duration']

0          C#3.G#3.A4.C#5@0.3
1              A3.C#5.E5@1.25
2               A2.B4.D5@1.25
3         E3.B4.C#5.D5.E5@0.3
4                E3.B4.D5@0.3
                 ...         
219299                   C4@2
219300                 C4@0.3
219301               B-3@1.25
219302             A3.B-3@0.3
219303                   A3@2
Name: target_chords_and_duration, Length: 219304, dtype: object

In [64]:
x=df['32_chords_and_duration']
y=df['target_chords_and_duration']

In [65]:
x

0         ['D2.D3@1.25' 'D2.D3.A3@0.3' 'D2.D3.D4.F#4.A4@...
1         ['D2.D3.A3@0.3' 'D2.D3.D4.F#4.A4@1.25' 'A1.A2....
2         ['D2.D3.D4.F#4.A4@1.25' 'A1.A2.D4.F#4.A4@1.25'...
3         ['A1.A2.D4.F#4.A4@1.25' 'D2.D3@1.25' 'D2.D3.A3...
4         ['D2.D3@1.25' 'D2.D3.A3@0.3' 'D2.D3.D4.F#4.A4@...
                                ...                        
219299    ['D2.A2.F3.A3.D4@1.25' 'D3.E3.D4@1.25' 'D3.E3....
219300    ['D3.E3.D4@1.25' 'D3.E3.G3.D4@1.25' 'D3.E3.A3....
219301    ['D3.E3.G3.D4@1.25' 'D3.E3.A3.D4@1.25' 'D3.E3....
219302    ['D3.E3.A3.D4@1.25' 'D3.E3.B-3.D4@1.25' 'D2.A2...
219303    ['D3.E3.B-3.D4@1.25' 'D2.A2.B-3@1.25' 'D2.A2.F...
Name: 32_chords_and_duration, Length: 219304, dtype: object

In [66]:
import numpy as np
x=np.array(x)
y=np.array(y)


In [67]:
y.shape

(219304,)

In [68]:
x[0]

"['D2.D3@1.25' 'D2.D3.A3@0.3' 'D2.D3.D4.F#4.A4@1.25' 'A1.A2.D4.F#4.A4@1.25'\n 'D2.D3@1.25' 'D2.D3.A3@0.3' 'D2.D3.D4.F#4.A4@1.25' 'A1.A2.D4.F#4.A4@1.25'\n 'D3.A3.D4.F#4@1.25' 'D3.A3.D4.F#4@1.25' 'D3.A3.D4.F#4@0.1'\n 'D3.G#3.F#4@0.3' 'D3.G#3@0.1' 'A3.G4.A4@0.3' 'A3.G4@0.3' 'A2.F#4@1.25'\n 'D3.A3.D4.F#4.A4@1.25' 'D3.G#3.A3.D4.F#4.A4@0.3' 'A3.D4.F#4.A4@1.25'\n 'A2.F#4.D5@0.3' 'A2.E5@0.3' 'D3.F#4.F#5@1.25' 'D3.G#3.F#4.C#5.E5@0.3'\n 'A3.F#4.D5@1.25' 'E-3.F4.B4@0.3' 'E-3.F4.F#4.B4@0.3' 'E3.C#4.G4@1.25'\n 'E3.G#3.C#4.G4@0.3' 'A3.C#4.G4@1.25' 'A2.E4.G4@0.3' 'A2.F#4.A4@0.3'\n 'C#3.G4.B4@1.25']"

In [69]:

len(x)

219304

In [70]:
import numpy as np
temp_x = []
str = "['A3.A5@1.25' 'E4.E6@2' 'E4.E6@1.25' 'D4.E4.D6.E6@0.3' 'D4.D6@1.25'\n 'C4.C6@1.25' 'D4.D6@1.25' 'E4.E6@1.25' 'F4.F6@1.25' 'G4.G6@1.25']"

for i in range(len(x)):
    str = x[i]
    # Remove square brackets and "\n" characters
    str = str.replace("[", "").replace("]", "").replace("\n", "").replace("'","")
    

    # Split the string by space to get a list of strings
    str_list = str.split(" ")
    

    # Convert the list of strings to a numpy array
    result = np.array(str_list)
    temp_x.append(result)
    
    ##############################################################
    # to calculate progress:
    total_length = len(x)
    iterative = i+1
    percent = ((i+1)/len(x)) * 100
    if int(percent) % 5 ==0 :
        print( percent,"% done")
    #############################################################
    
    #print(result, "type = ", type(result))
    #np.vstack([temp_x,result])


0.00045598803487396493 % done
0.0009119760697479299 % done
0.0013679641046218946 % done
0.0018239521394958597 % done
0.0022799401743698244 % done
0.0027359282092437893 % done
0.0031919162441177546 % done
0.0036479042789917195 % done
0.004103892313865684 % done
0.004559880348739649 % done
0.005015868383613614 % done
0.0054718564184875786 % done
0.005927844453361544 % done
0.006383832488235509 % done
0.006839820523109474 % done
0.007295808557983439 % done
0.007751796592857404 % done
0.008207784627731368 % done
0.008663772662605334 % done
0.009119760697479298 % done
0.009575748732353263 % done
0.010031736767227227 % done
0.010487724802101193 % done
0.010943712836975157 % done
0.011399700871849123 % done
0.011855688906723089 % done
0.012311676941597051 % done
0.012767664976471018 % done
0.01322365301134498 % done
0.013679641046218948 % done
0.01413562908109291 % done
0.014591617115966878 % done
0.01504760515084084 % done
0.015503593185714808 % done
0.015959581220588773 % done
0.01641556925

5.000364790427899 % done
5.000820778462773 % done
5.0012767664976465 % done
5.0017327545325205 % done
5.0021887425673945 % done
5.0026447306022686 % done
5.003100718637143 % done
5.003556706672017 % done
5.004012694706891 % done
5.004468682741765 % done
5.004924670776639 % done
5.005380658811513 % done
5.005836646846387 % done
5.006292634881261 % done
5.006748622916135 % done
5.007204610951009 % done
5.007660598985883 % done
5.008116587020757 % done
5.00857257505563 % done
5.009028563090504 % done
5.009484551125378 % done
5.009940539160253 % done
5.010396527195127 % done
5.010852515230001 % done
5.011308503264875 % done
5.011764491299749 % done
5.012220479334623 % done
5.012676467369497 % done
5.013132455404371 % done
5.013588443439245 % done
5.014044431474119 % done
5.014500419508992 % done
5.014956407543866 % done
5.01541239557874 % done
5.015868383613614 % done
5.016324371648488 % done
5.016780359683362 % done
5.017236347718236 % done
5.01769233575311 % done
5.018148323787984 % done

10.000273592820925 % done
10.000729580855799 % done
10.001185568890673 % done
10.001641556925547 % done
10.00209754496042 % done
10.002553532995293 % done
10.003009521030167 % done
10.003465509065041 % done
10.003921497099915 % done
10.004377485134789 % done
10.004833473169663 % done
10.005289461204537 % done
10.005745449239411 % done
10.006201437274285 % done
10.00665742530916 % done
10.007113413344033 % done
10.007569401378907 % done
10.008025389413781 % done
10.008481377448655 % done
10.00893736548353 % done
10.009393353518403 % done
10.009849341553277 % done
10.010305329588151 % done
10.010761317623025 % done
10.0112173056579 % done
10.011673293692773 % done
10.012129281727647 % done
10.012585269762521 % done
10.013041257797395 % done
10.01349724583227 % done
10.013953233867143 % done
10.014409221902017 % done
10.014865209936891 % done
10.015321197971765 % done
10.01577718600664 % done
10.016233174041513 % done
10.016689162076386 % done
10.01714515011126 % done
10.017601138146134 %

15.00018239521395 % done
15.000638383248823 % done
15.001094371283695 % done
15.001550359318571 % done
15.002006347353444 % done
15.00246233538832 % done
15.002918323423192 % done
15.003374311458067 % done
15.00383029949294 % done
15.004286287527815 % done
15.004742275562688 % done
15.005198263597563 % done
15.005654251632436 % done
15.006110239667311 % done
15.006566227702184 % done
15.00702221573706 % done
15.007478203771932 % done
15.007934191806807 % done
15.00839017984168 % done
15.008846167876555 % done
15.009302155911428 % done
15.009758143946303 % done
15.010214131981176 % done
15.010670120016051 % done
15.011126108050924 % done
15.0115820960858 % done
15.012038084120672 % done
15.012494072155548 % done
15.01295006019042 % done
15.013406048225294 % done
15.01386203626017 % done
15.014318024295042 % done
15.014774012329918 % done
15.01523000036479 % done
15.015685988399666 % done
15.016141976434538 % done
15.016597964469414 % done
15.017053952504286 % done
15.017509940539162 % d

20.000091197606977 % done
20.00054718564185 % done
20.00100317367672 % done
20.001459161711598 % done
20.00191514974647 % done
20.002371137781346 % done
20.002827125816218 % done
20.003283113851094 % done
20.003739101885966 % done
20.00419508992084 % done
20.004651077955714 % done
20.005107065990586 % done
20.005563054025462 % done
20.006019042060334 % done
20.00647503009521 % done
20.006931018130082 % done
20.007387006164958 % done
20.00784299419983 % done
20.008298982234706 % done
20.008754970269578 % done
20.009210958304454 % done
20.009666946339326 % done
20.010122934374202 % done
20.010578922409074 % done
20.01103491044395 % done
20.011490898478822 % done
20.011946886513698 % done
20.01240287454857 % done
20.012858862583446 % done
20.01331485061832 % done
20.013770838653194 % done
20.014226826688066 % done
20.014682814722942 % done
20.015138802757814 % done
20.01559479079269 % done
20.016050778827562 % done
20.016506766862438 % done
20.01696275489731 % done
20.017418742932186 % do

...

80.00036479042791 % done
80.00082077846278 % done
80.00127676649764 % done
80.00173275453251 % done
80.0021887425674 % done
80.00264473060227 % done
80.00310071863714 % done
80.00355670667201 % done
80.00401269470689 % done
80.00446868274177 % done
80.00492467077663 % done
80.0053806588115 % done
80.00583664684639 % done
80.00629263488126 % done
80.00674862291613 % done
80.007204610951 % done
80.00766059898588 % done
80.00811658702077 % done
80.00857257505562 % done
80.0090285630905 % done
80.00948455112538 % done
80.00994053916025 % done
80.01039652719513 % done
80.01085251523 % done
80.01130850326487 % done
80.01176449129976 % done
80.01222047933462 % done
80.01267646736949 % done
80.01313245540437 % done
80.01358844343925 % done
80.01404443147412 % done
80.01450041950899 % done
80.01495640754386 % done
80.01541239557875 % done
80.01586838361361 % done
80.01632437164848 % done
80.01678035968337 % done
80.01723634771824 % done
80.01769233575311 % done
80.01814832378798 % done
80.01860

85.00027359282093 % done
85.00072958085579 % done
85.00118556889066 % done
85.00164155692555 % done
85.00209754496042 % done
85.0025535329953 % done
85.00300952103017 % done
85.00346550906505 % done
85.00392149709992 % done
85.00437748513478 % done
85.00483347316965 % done
85.00528946120454 % done
85.00574544923941 % done
85.00620143727429 % done
85.00665742530916 % done
85.00711341334404 % done
85.00756940137892 % done
85.00802538941377 % done
85.00848137744865 % done
85.00893736548353 % done
85.0093933535184 % done
85.00984934155328 % done
85.01030532958815 % done
85.01076131762304 % done
85.01121730565791 % done
85.01167329369277 % done
85.01212928172764 % done
85.01258526976252 % done
85.0130412577974 % done
85.01349724583227 % done
85.01395323386714 % done
85.01440922190201 % done
85.0148652099369 % done
85.01532119797176 % done
85.01577718600663 % done
85.01623317404152 % done
85.01668916207639 % done
85.01714515011126 % done
85.01760113814613 % done
85.018057126181 % done
85.018

90.00018239521394 % done
90.00063838324883 % done
90.0010943712837 % done
90.00155035931857 % done
90.00200634735344 % done
90.00246233538832 % done
90.0029183234232 % done
90.00337431145806 % done
90.00383029949293 % done
90.00428628752782 % done
90.00474227556269 % done
90.00519826359756 % done
90.00565425163244 % done
90.00611023966731 % done
90.0065662277022 % done
90.00702221573705 % done
90.00747820377192 % done
90.00793419180681 % done
90.00839017984168 % done
90.00884616787656 % done
90.00930215591143 % done
90.0097581439463 % done
90.01021413198119 % done
90.01067012001604 % done
90.01112610805092 % done
90.01158209608579 % done
90.01203808412068 % done
90.01249407215555 % done
90.01295006019042 % done
90.01340604822529 % done
90.01386203626018 % done
90.01431802429505 % done
90.01477401232991 % done
90.01523000036478 % done
90.01568598839967 % done
90.01614197643454 % done
90.01659796446941 % done
90.01705395250428 % done
90.01750994053917 % done
90.01796592857404 % done
90.0

95.00009119760698 % done
95.00054718564185 % done
95.00100317367672 % done
95.0014591617116 % done
95.00191514974648 % done
95.00237113778135 % done
95.00282712581621 % done
95.00328311385108 % done
95.00373910188597 % done
95.00419508992084 % done
95.00465107795571 % done
95.00510706599059 % done
95.00556305402547 % done
95.00601904206034 % done
95.0064750300952 % done
95.00693101813008 % done
95.00738700616496 % done
95.00784299419983 % done
95.0082989822347 % done
95.00875497026958 % done
95.00921095830446 % done
95.00966694633934 % done
95.0101229343742 % done
95.01057892240907 % done
95.01103491044395 % done
95.01149089847883 % done
95.0119468865137 % done
95.01240287454857 % done
95.01285886258344 % done
95.01331485061833 % done
95.01377083865319 % done
95.01422682668806 % done
95.01468281472295 % done
95.01513880275782 % done
95.01559479079269 % done
95.01605077882756 % done
95.01650676686243 % done
95.01696275489732 % done
95.01741874293218 % done
95.01787473096705 % done
95.01

100.0 % done


In [71]:
#convert list to array:
x = np.array(temp_x)

In [72]:
type(x)

numpy.ndarray

In [73]:
x

array([['D2.D3@1.25', 'D2.D3.A3@0.3', 'D2.D3.D4.F#4.A4@1.25', ...,
        'A2.E4.G4@0.3', 'A2.F#4.A4@0.3', 'C#3.G4.B4@1.25'],
       ['D2.D3.A3@0.3', 'D2.D3.D4.F#4.A4@1.25', 'A1.A2.D4.F#4.A4@1.25',
        ..., 'A2.F#4.A4@0.3', 'C#3.G4.B4@1.25', 'C#3.G#3.A4.C#5@0.3'],
       ['D2.D3.D4.F#4.A4@1.25', 'A1.A2.D4.F#4.A4@1.25', 'D2.D3@1.25',
        ..., 'C#3.G4.B4@1.25', 'C#3.G#3.A4.C#5@0.3', 'A3.C#5.E5@1.25'],
       ...,
       ['D3.E3.G3.D4@1.25', 'D3.E3.A3.D4@1.25', 'D3.E3.B-3.D4@1.25', ...,
        'A3@0.3', 'C4@2', 'C4@0.3'],
       ['D3.E3.A3.D4@1.25', 'D3.E3.B-3.D4@1.25', 'D2.A2.B-3@1.25', ...,
        'C4@2', 'C4@0.3', 'B-3@1.25'],
       ['D3.E3.B-3.D4@1.25', 'D2.A2.B-3@1.25', 'D2.A2.F3@1.25', ...,
        'C4@0.3', 'B-3@1.25', 'A3.B-3@0.3']], dtype='<U50')

In [74]:
y[0]

'C#3.G#3.A4.C#5@0.3'

In [75]:
#assigning unique integer to every chords_and_duration

unique_x_CD = list(set(x.ravel()))
unique_x_CD_to_int = dict((chord_and_duration, number) for number, chord_and_duration in enumerate(unique_x_CD))
len(unique_x_CD_to_int)

49645

In [76]:
unique_x_CD_to_int

{'E-3.D5@0.3': 0,
 'C3.E3.G#4@1.25': 1,
 'F2.F3.D4.F4.C5@0.3': 2,
 'B-3.A4.G5.A5@0.3': 3,
 'E2.E3.C#5.G5.B-5@1.25': 4,
 'E3.F#3.B-3.C#4.C#5.C#6@0.3': 5,
 'E-4.G4.B-4.F5.E-6@0.3': 6,
 'F4.G4.F5.B-5.D6.F6@0.1': 7,
 'D2.F#2.A2.C3.F5.F#5.G#5.A5@0.3': 8,
 'G2.B2.D3.G3.B3.D4.G4@1.25': 9,
 'G2.E-4.B-4@0.3': 10,
 'E2.E3.B-4.C5.G5.B-5@1.25': 11,
 'A1.D5.F#5@1.25': 12,
 'F3.G#3.F4@0.1': 13,
 'G2.D3.G3.B-3.D4.G4.B-4@1.25': 14,
 'F#2.D3.B-3.B-4@1.25': 15,
 'E-2.G#2.E-3.G#4.B4.F5.B5@0.3': 16,
 'D3.G#3.F4@1.25': 17,
 'D2.E3.G#3.D5.F5.D6@0.1': 18,
 'D4.F5.F6@0.3': 19,
 'C3.E-5.G5.E-6@1.25': 20,
 'G2.B-3.C#4.E4@0.3': 21,
 'F2.A4.C5@1.25': 22,
 'A4.F6@0.3': 23,
 'B-3.E4.C5.A5@1.25': 24,
 'E4.C5.E5@1.25': 25,
 'G#2.F#3.D4.F#4@1.25': 26,
 'A1.A3.G4.C5@0.3': 27,
 'C#3.C#4.A4.C#5@1.25': 28,
 'C4.E-4.G4.C5@0.1': 29,
 'C3.G#4.C5@0.3': 30,
 'E-2.G#2.E-3.G#3.B3.G4.B-4.B4.F5.B5@0.1': 31,
 'G#3.F#6@0.3': 32,
 'E3.A4.C#5@0.1': 33,
 'D4.F#4.E-5.G5@0.3': 34,
 'E-4.E-5.G#5@0.1': 35,
 'E-7.E7@0.1': 36,
 'G2.D3.G3.B3.

In [77]:
#preparing input sequences::

x_seq=[]
for each_row in x:
    temp=[]
    for each_piece in each_row:
        #assigning unique integer to every note
        temp.append(unique_x_CD_to_int[each_piece])
    x_seq.append(temp)
    
x_seq = np.array(x_seq)
x_seq.shape

(219304, 32)

In [78]:
# preparing th output sequences as well::

unique_y_CD = list(set(y))
unique_y_CD_to_int = dict((chord_and_duration, number) for number, chord_and_duration in enumerate(unique_y_CD)) 
unique_y_CD_to_int

{'E-3.D5@0.3': 0,
 'C3.E3.G#4@1.25': 1,
 'F2.F3.D4.F4.C5@0.3': 2,
 'B-3.A4.G5.A5@0.3': 3,
 'E2.E3.C#5.G5.B-5@1.25': 4,
 'E3.F#3.B-3.C#4.C#5.C#6@0.3': 5,
 'E-4.G4.B-4.F5.E-6@0.3': 6,
 'F4.G4.F5.B-5.D6.F6@0.1': 7,
 'D2.F#2.A2.C3.F5.F#5.G#5.A5@0.3': 8,
 'G2.B2.D3.G3.B3.D4.G4@1.25': 9,
 'G2.E-4.B-4@0.3': 10,
 'E2.E3.B-4.C5.G5.B-5@1.25': 11,
 'A1.D5.F#5@1.25': 12,
 'F3.G#3.F4@0.1': 13,
 'G2.D3.G3.B-3.D4.G4.B-4@1.25': 14,
 'F#2.D3.B-3.B-4@1.25': 15,
 'E-2.G#2.E-3.G#4.B4.F5.B5@0.3': 16,
 'D3.G#3.F4@1.25': 17,
 'D2.E3.G#3.D5.F5.D6@0.1': 18,
 'D4.F5.F6@0.3': 19,
 'C3.E-5.G5.E-6@1.25': 20,
 'G2.B-3.C#4.E4@0.3': 21,
 'F2.A4.C5@1.25': 22,
 'A4.F6@0.3': 23,
 'B-3.E4.C5.A5@1.25': 24,
 'E4.C5.E5@1.25': 25,
 'G#2.F#3.D4.F#4@1.25': 26,
 'A1.A3.G4.C5@0.3': 27,
 'C#3.C#4.A4.C#5@1.25': 28,
 'C4.E-4.G4.C5@0.1': 29,
 'C3.G#4.C5@0.3': 30,
 'E-2.G#2.E-3.G#3.B3.G4.B-4.B4.F5.B5@0.1': 31,
 'G#3.F#6@0.3': 32,
 'E3.A4.C#5@0.1': 33,
 'D4.F#4.E-5.G5@0.3': 34,
 'E-4.E-5.G#5@0.1': 35,
 'E-7.E7@0.1': 36,
 'G2.D3.G3.B3.

In [79]:
len(unique_y_CD_to_int)

49370

In [80]:
y_seq=np.array([unique_y_CD_to_int[i] for i in y])
y_seq.shape

(219304,)

In [81]:
# preserving 80% of the data for training and the rest 20% for the evaluation:

from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_seq,y_seq,test_size=0.2,random_state=0)

In [82]:
len(x_val)

43861

# model building phase:

In [83]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [84]:
from keras.layers import *
from keras.models import *
from keras.callbacks import *
import keras.backend as K

K.clear_session()
model = Sequential()
    
#embedding layer
model.add(Embedding(len(unique_x_CD), 100, input_length=32,trainable=True)) 

model.add(Conv1D(64,3, padding='causal',activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))
    
model.add(Conv1D(128,3,activation='relu',dilation_rate=2,padding='causal'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))

model.add(Conv1D(256,3,activation='relu',dilation_rate=4,padding='causal'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))
          
#model.add(Conv1D(256,5,activation='relu'))    
model.add(GlobalMaxPool1D())
    
model.add(Dense(256, activation='relu'))
model.add(Dense(len(unique_y_CD), activation='softmax'))
    
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 100)           4964500   
                                                                 
 conv1d (Conv1D)             (None, 32, 64)            19264     
                                                                 
 dropout (Dropout)           (None, 32, 64)            0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 16, 64)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 16, 128)           24704     
                                                                 
 dropout_1 (Dropout)         (None, 16, 128)           0         
                                                        

In [85]:
def lstm():
    K.clear_session()
    model = Sequential()
    #embedding layer
    model.add(Embedding(len(unique_x_CD), 100, input_length=32,trainable=True)) 
    model.add(LSTM(128,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(128))
    model.add(Dropout(0.2))
    model.add(Dense(256))
    model.add(Activation('relu'))

    model.add(Dense(len(unique_y_CD), activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
    return model

In [86]:
model = lstm()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 100)           4964500   
                                                                 
 lstm (LSTM)                 (None, 32, 128)           117248    
                                                                 
 dropout (Dropout)           (None, 32, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 256)               33024     
                                                                 
 activation (Activation)     (None, 256)               0

In [87]:
!pip install h5py

In [88]:
import h5py

In [89]:
#defining call back to save the best model during training>
mc=ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True,verbose=1)

In [90]:
#actual training
#history = model.fit(np.array(x_train),np.array(y_train),batch_size=128,epochs=10, 
                   #validation_data=(np.array(x_val),np.array(y_val)),verbose=1, callbacks=[mc])

In [91]:
#loading best model (Previously trained modle)
from keras.models import load_model
model = load_model('../models/best_model_major_dataset_295_songs.h5')


In [92]:
import numpy as np
import random
ind = np.random.randint(0,len(x_val)-1)
random_music = x_val[ind]
random_music

array([ 9532,  3428, 32656,  1944, 13467, 15126, 36762, 43057, 22027,
       43057, 36762, 43057, 22027, 43057, 36762, 43057, 22027, 43057,
        3465,  3036, 39902, 23617,  3465,  7301, 39648,  7301, 13183,
        7894, 40529,  7894, 47078,  7987])

In [93]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [94]:
no_of_timesteps = 32

In [110]:
predictions=[]
for i in range(100):

    random_music = random_music.reshape(1,no_of_timesteps)
    print("random music = ", random_music)
    

    prob  = model.predict(random_music)[0]
    y_pred= np.argmax(prob,axis=0)
    predictions.append(y_pred)

    random_music = np.insert(random_music[0],len(random_music[0]),y_pred)
    random_music = random_music[1:]
    
print(predictions)

random music =  [[19336 32977 45655 19994 45780 15642 46653 19994 45780  7096 23467 47156
   7096 47156  7096 47156  7096 47156  7096 47156  7096 47156  7096 47156
   7096 47156  7096 47156  7096 47156  7096 47156]]
random music =  [[32977 45655 19994 45780 15642 46653 19994 45780  7096 23467 47156  7096
  47156  7096 47156  7096 47156  7096 47156  7096 47156  7096 47156  7096
  47156  7096 47156  7096 47156  7096 47156  7096]]
random music =  [[45655 19994 45780 15642 46653 19994 45780  7096 23467 47156  7096 47156
   7096 47156  7096 47156  7096 47156  7096 47156  7096 47156  7096 47156
   7096 47156  7096 47156  7096 47156  7096 47156]]
random music =  [[19994 45780 15642 46653 19994 45780  7096 23467 47156  7096 47156  7096
  47156  7096 47156  7096 47156  7096 47156  7096 47156  7096 47156  7096
  47156  7096 47156  7096 47156  7096 47156  7096]]
random music =  [[45780 15642 46653 19994 45780  7096 23467 47156  7096 47156  7096 47156
   7096 47156  7096 47156  7096 47156  7096 47

random music =  [[47156 38978 48582 44917 46302 31972  3512 31972  3512 34842  5913 41526
  31972  3512 34842  5913 41526 31972  3512 31972  3512 34175  7623  8114
   3512  5677 48873 32023 20446 26290 14971 16012]]
random music =  [[38978 48582 44917 46302 31972  3512 31972  3512 34842  5913 41526 31972
   3512 34842  5913 41526 31972  3512 31972  3512 34175  7623  8114  3512
   5677 48873 32023 20446 26290 14971 16012 34727]]
random music =  [[48582 44917 46302 31972  3512 31972  3512 34842  5913 41526 31972  3512
  34842  5913 41526 31972  3512 31972  3512 34175  7623  8114  3512  5677
  48873 32023 20446 26290 14971 16012 34727 14971]]
random music =  [[44917 46302 31972  3512 31972  3512 34842  5913 41526 31972  3512 34842
   5913 41526 31972  3512 31972  3512 34175  7623  8114  3512  5677 48873
  32023 20446 26290 14971 16012 34727 14971 39186]]
random music =  [[46302 31972  3512 31972  3512 34842  5913 41526 31972  3512 34842  5913
  41526 31972  3512 31972  3512 34175  7623  8

random music =  [[41526 47156 38978 47504 38185 14758  7096 47156  7096 47156  7096 47156
   7096 47156  7096 47156  7096 47156  7096 47156  7096 47156  7096 47156
   7096 47156  7096 47156 38978 30211 32689 30574]]
random music =  [[47156 38978 47504 38185 14758  7096 47156  7096 47156  7096 47156  7096
  47156  7096 47156  7096 47156  7096 47156  7096 47156  7096 47156  7096
  47156  7096 47156 38978 30211 32689 30574 15056]]
random music =  [[38978 47504 38185 14758  7096 47156  7096 47156  7096 47156  7096 47156
   7096 47156  7096 47156  7096 47156  7096 47156  7096 47156  7096 47156
   7096 47156 38978 30211 32689 30574 15056 48582]]
random music =  [[47504 38185 14758  7096 47156  7096 47156  7096 47156  7096 47156  7096
  47156  7096 47156  7096 47156  7096 47156  7096 47156  7096 47156  7096
  47156 38978 30211 32689 30574 15056 48582 44917]]
random music =  [[38185 14758  7096 47156  7096 47156  7096 47156  7096 47156  7096 47156
   7096 47156  7096 47156  7096 47156  7096 47

In [111]:
#intergers back to notes
unique_y_int_to_CD = dict((number, note_) for number, note_ in enumerate(unique_y_CD)) 
unique_y_int_to_CD

{0: 'E-3.D5@0.3',
 1: 'C3.E3.G#4@1.25',
 2: 'F2.F3.D4.F4.C5@0.3',
 3: 'B-3.A4.G5.A5@0.3',
 4: 'E2.E3.C#5.G5.B-5@1.25',
 5: 'E3.F#3.B-3.C#4.C#5.C#6@0.3',
 6: 'E-4.G4.B-4.F5.E-6@0.3',
 7: 'F4.G4.F5.B-5.D6.F6@0.1',
 8: 'D2.F#2.A2.C3.F5.F#5.G#5.A5@0.3',
 9: 'G2.B2.D3.G3.B3.D4.G4@1.25',
 10: 'G2.E-4.B-4@0.3',
 11: 'E2.E3.B-4.C5.G5.B-5@1.25',
 12: 'A1.D5.F#5@1.25',
 13: 'F3.G#3.F4@0.1',
 14: 'G2.D3.G3.B-3.D4.G4.B-4@1.25',
 15: 'F#2.D3.B-3.B-4@1.25',
 16: 'E-2.G#2.E-3.G#4.B4.F5.B5@0.3',
 17: 'D3.G#3.F4@1.25',
 18: 'D2.E3.G#3.D5.F5.D6@0.1',
 19: 'D4.F5.F6@0.3',
 20: 'C3.E-5.G5.E-6@1.25',
 21: 'G2.B-3.C#4.E4@0.3',
 22: 'F2.A4.C5@1.25',
 23: 'A4.F6@0.3',
 24: 'B-3.E4.C5.A5@1.25',
 25: 'E4.C5.E5@1.25',
 26: 'G#2.F#3.D4.F#4@1.25',
 27: 'A1.A3.G4.C5@0.3',
 28: 'C#3.C#4.A4.C#5@1.25',
 29: 'C4.E-4.G4.C5@0.1',
 30: 'C3.G#4.C5@0.3',
 31: 'E-2.G#2.E-3.G#3.B3.G4.B-4.B4.F5.B5@0.1',
 32: 'G#3.F#6@0.3',
 33: 'E3.A4.C#5@0.1',
 34: 'D4.F#4.E-5.G5@0.3',
 35: 'E-4.E-5.G#5@0.1',
 36: 'E-7.E7@0.1',
 37: 'G2.D3.G3

In [112]:
predicted_CD = [unique_y_int_to_CD[i] for i in predictions]
predicted_CD

['B2.G#4.B4.E5@0.3',
 'F#2.F4.B-4@0.1',
 'B2.G#4.B4.E5@0.3',
 'F#2.F4.B-4@0.1',
 'B2.G#4.B4.E5@0.3',
 'F#2.F4.B-4@0.1',
 'C4.F#4.A4.C5.D5@0.3',
 'F#2.F4.B-4@0.1',
 'A2.C3@0.1',
 'F1.F2.E-7@0.3',
 'G#1.G#2.B-2@1.25',
 'D2.D4.B-4.D5@0.3',
 'E-3.F#3@0.1',
 'E-3.C4.G#4.E-5@1.25',
 'E-3.F#3@0.1',
 'E-3.C4.G#4.E-5@1.25',
 'E-3.F4.G#4.F5@0.3',
 'C5.C#5.B5.E-6.B6@0.3',
 'B2.E-6.B-6@0.3',
 'E-3.F#3@0.1',
 'E-3.C4.G#4.E-5@1.25',
 'E-3.F4.G#4.F5@0.3',
 'C5.C#5.B5.E-6.B6@0.3',
 'B2.E-6.B-6@0.3',
 'E-3.F#3@0.1',
 'E-3.C4.G#4.E-5@1.25',
 'E-3.F#3@0.1',
 'E-3.C4.G#4.E-5@1.25',
 'D3.F#3.D4.C5.D5@0.3',
 'E-4.E-5.D6.G6@0.3',
 'G3.D4.G4@0.1',
 'E-3.C4.G#4.E-5@1.25',
 'C#2.C#3.E-4.G4.B-4@1.25',
 'F3.E4.G#4.C5@0.3',
 'F2.A6.E7@0.3',
 'D2.D3.F#3.D4.F#4@1.25',
 'G#3.A3.B3@0.3',
 'G4.D5.C6@1.25',
 'F#4.B4.B5@0.1',
 'F2.F4.E-5@0.3',
 'G4.D5.C6@1.25',
 'D4.B4.F5@0.3',
 'A2.E-3.F3.B3.B4@0.3',
 'A2.E-3.F3.B3.B4@0.3',
 'F2.D4@0.3',
 'B2.E-6.B-6@0.3',
 'F#2.F4.B-4@0.1',
 'A2.C3@0.1',
 'C#4.E-5.F5.B-5@0.1',
 'D5.A5.

In [113]:
predicted_CD_split = []
for each_outcome in predicted_CD:
    temp_list = []
    temp_list = each_outcome.split("@")
    temp_list[1] =float(temp_list[1])
    predicted_CD_split.append(tuple(temp_list))
    

predicted_CD_split

[('B2.G#4.B4.E5', 0.3),
 ('F#2.F4.B-4', 0.1),
 ('B2.G#4.B4.E5', 0.3),
 ('F#2.F4.B-4', 0.1),
 ('B2.G#4.B4.E5', 0.3),
 ('F#2.F4.B-4', 0.1),
 ('C4.F#4.A4.C5.D5', 0.3),
 ('F#2.F4.B-4', 0.1),
 ('A2.C3', 0.1),
 ('F1.F2.E-7', 0.3),
 ('G#1.G#2.B-2', 1.25),
 ('D2.D4.B-4.D5', 0.3),
 ('E-3.F#3', 0.1),
 ('E-3.C4.G#4.E-5', 1.25),
 ('E-3.F#3', 0.1),
 ('E-3.C4.G#4.E-5', 1.25),
 ('E-3.F4.G#4.F5', 0.3),
 ('C5.C#5.B5.E-6.B6', 0.3),
 ('B2.E-6.B-6', 0.3),
 ('E-3.F#3', 0.1),
 ('E-3.C4.G#4.E-5', 1.25),
 ('E-3.F4.G#4.F5', 0.3),
 ('C5.C#5.B5.E-6.B6', 0.3),
 ('B2.E-6.B-6', 0.3),
 ('E-3.F#3', 0.1),
 ('E-3.C4.G#4.E-5', 1.25),
 ('E-3.F#3', 0.1),
 ('E-3.C4.G#4.E-5', 1.25),
 ('D3.F#3.D4.C5.D5', 0.3),
 ('E-4.E-5.D6.G6', 0.3),
 ('G3.D4.G4', 0.1),
 ('E-3.C4.G#4.E-5', 1.25),
 ('C#2.C#3.E-4.G4.B-4', 1.25),
 ('F3.E4.G#4.C5', 0.3),
 ('F2.A6.E7', 0.3),
 ('D2.D3.F#3.D4.F#4', 1.25),
 ('G#3.A3.B3', 0.3),
 ('G4.D5.C6', 1.25),
 ('F#4.B4.B5', 0.1),
 ('F2.F4.E-5', 0.3),
 ('G4.D5.C6', 1.25),
 ('D4.B4.F5', 0.3),
 ('A2.E-3.F3.B3.B4'

In [114]:
print("Generating music from our processed chords...")
proccessed_chords_to_midi_sample = get_music_midi_from_chords_and_duration(predicted_CD_split)
print(proccessed_chords_to_midi_sample)
play_midi_file(proccessed_chords_to_midi_sample)

Generating music from our processed chords...
output-20230201-090628.mid


In [105]:
print("Generating music from our processed chords...")
proccessed_chords_to_midi_sample = get_music_midi_from_chords_and_duration(predicted_CD_split)
print(proccessed_chords_to_midi_sample)
play_midi_file(proccessed_chords_to_midi_sample)

Generating music from our processed chords...
output-20230201-085756.mid
